# Chats and memory with langchain

In [1]:
import numpy as np
import pandas as pd

In [2]:
from langchain.chat_models import ChatOpenAI

Load the OPEN AI KEY

In [3]:
# load openai key
with open('../openaikey.txt', "r") as file:
    # Read the contents of the file
    OPENAI_KEY = file.read()

In [4]:
llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model_name='gpt-3.5-turbo', temperature=0, max_tokens=250)

Import libraries for chat schemas

In [5]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [6]:
message = [
    SystemMessage(content='Hello, I am a chatbot. I am here to help you with your questions.'),
    HumanMessage(content='Hello, I have a question about my order.'),
    AIMessage(content='Yes, go ahead and tell me your question.'),
    HumanMessage(content='How much is 2 + 2?'),
]

In [7]:
res = llm(message)

In [8]:
res

AIMessage(content='The sum of 2 + 2 is 4.', additional_kwargs={}, example=False)

Continue with the chat

In [9]:
message.append(res)

Add a new question

In [10]:
message.append(HumanMessage(content='Ok, thanks!, now tell me what is the difference between Canada and USA'))

New answer

In [11]:
res = llm(message)

In [12]:
res

AIMessage(content='Canada and the United States are neighboring countries in North America, but there are several differences between them. Here are a few key distinctions:\n\n1. Government and Political Systems: Canada is a parliamentary democracy with a constitutional monarchy, where the British monarch is the head of state. The United States, on the other hand, is a federal republic with a presidential system.\n\n2. Healthcare: Canada has a publicly funded healthcare system known as Medicare, which provides universal healthcare coverage to all Canadian citizens and permanent residents. In the United States, healthcare is primarily provided through private insurance, and access to healthcare can vary depending on individual insurance coverage.\n\n3. Education: Both countries have strong education systems, but there are some differences. In Canada, education is primarily a provincial responsibility, resulting in some variations across provinces. In the United States, education is prim

## Chat templates

They´re useful when it is needed to add information in a prompt

In [15]:
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [24]:
template_system = """
You are an expert to answer questions about {brand}.
"""

In [25]:
prompt_system = SystemMessagePromptTemplate.from_template(template_system)

In [26]:
template_human = """ 
{question}
"""

In [27]:
promt_human = HumanMessagePromptTemplate.from_template(template_human)

In [31]:
prompt_chat = ChatPromptTemplate.from_messages([template_system, template_human])

In [32]:
prompt_chat_format = prompt_chat.format_prompt(brand = 'Apple', question = 'How much does it cost the newest phone')

In [34]:
llm(prompt_chat_format.to_messages())

AIMessage(content="As an AI language model, I don't have real-time information on the latest Apple products and their prices. However, as of my last update, the newest iPhone models released by Apple are typically priced starting from around $699 for the base model. However, prices can vary depending on the specific model, storage capacity, and any additional features or accessories. It's always best to check Apple's official website or authorized retailers for the most accurate and up-to-date pricing information.", additional_kwargs={}, example=False)

## Memory

In [13]:
from langchain.chains.conversation.memory import ConversationBufferMemory 
from langchain.chains import ConversationChain

1. ConversationBufferMemory is suitable in scenarios where the conversation is going to be less than 6 interactions

In [19]:
memory = ConversationBufferMemory()

In [20]:
conversation = ConversationChain(memory=memory, llm=llm, verbose=True)

In [21]:
conversation.predict(input='Quien descrubrio America?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Quien descrubrio America?
AI:

> Finished chain.


'Cristóbal Colón descubrió América en 1492.'

In [46]:
conversation.predict(input='Quien fundo la empresa Platzi?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Quien descrubrio America?
AI: Cristóbal Colón descubrió América en 1492. Fue un navegante y explorador genovés que realizó cuatro viajes transatlánticos en nombre de los Reyes Católicos de España. En su primer viaje, llegó a las islas del Caribe y posteriormente exploró las costas de América Central y del Sur. Su descubrimiento marcó el comienzo de la colonización europea en el continente americano.
Human: Quien fundo la empresa Platzi?
AI:

> Finished chain.


'La empresa Platzi fue fundada por Freddy Vega y Christian Van Der Henst en 2013. Platzi es una plataforma de educación en línea que ofrece cursos sobre programación, diseño, marketing y emprendimiento. La empresa tiene su sede en Bogotá, Colombia, y ha crecido rápidamente desde su fundación, convirtiéndose en una de las principales plataformas de educación en línea en América Latina.'

In [47]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Quien descrubrio America?', additional_kwargs={}, example=False),
 AIMessage(content='Cristóbal Colón descubrió América en 1492. Fue un navegante y explorador genovés que realizó cuatro viajes transatlánticos en nombre de los Reyes Católicos de España. En su primer viaje, llegó a las islas del Caribe y posteriormente exploró las costas de América Central y del Sur. Su descubrimiento marcó el comienzo de la colonización europea en el continente americano.', additional_kwargs={}, example=False),
 HumanMessage(content='Quien fundo la empresa Platzi?', additional_kwargs={}, example=False),
 AIMessage(content='La empresa Platzi fue fundada por Freddy Vega y Christian Van Der Henst en 2013. Platzi es una plataforma de educación en línea que ofrece cursos sobre programación, diseño, marketing y emprendimiento. La empresa tiene su sede en Bogotá, Colombia, y ha crecido rápidamente desde su fundación, convirtiéndose en una de las principales plataformas de educación en lí

2. Conversation window buffer memory

This type of memory is suitable when you want to remember something specifically from the conversation

In [64]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [65]:
window_memory = ConversationBufferWindowMemory(k=2)

In [66]:
conversation = ConversationChain(memory=window_memory, llm=llm, verbose=True)

In [67]:
conversation.predict(input='Hola quien fue Jose Maria Sarasti en Ecuador')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola quien fue Jose Maria Sarasti en Ecuador
AI:

> Finished chain.


'¡Hola! José María Sarasti fue un destacado político ecuatoriano. Nació el 19 de marzo de 1899 en Guayaquil, Ecuador, y falleció el 6 de julio de 1976 en la misma ciudad. Sarasti fue un miembro prominente del Partido Conservador y ocupó varios cargos políticos a lo largo de su carrera. Fue elegido como diputado en varias ocasiones y también se desempeñó como Ministro de Gobierno y Ministro de Relaciones Exteriores en diferentes períodos. Además, Sarasti fue conocido por su participación en la redacción de la Constitución de 1945 en Ecuador. ¿Hay algo más en lo que pueda ayudarte?'

In [68]:
conversation.predict(input='Cual fue la relacion de Jose Maria Sarasti con Eloy Alfaro')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola quien fue Jose Maria Sarasti en Ecuador
AI: ¡Hola! José María Sarasti fue un destacado político ecuatoriano. Nació el 19 de marzo de 1899 en Guayaquil, Ecuador, y falleció el 6 de julio de 1976 en la misma ciudad. Sarasti fue un miembro prominente del Partido Conservador y ocupó varios cargos políticos a lo largo de su carrera. Fue elegido como diputado en varias ocasiones y también se desempeñó como Ministro de Gobierno y Ministro de Relaciones Exteriores en diferentes períodos. Además, Sarasti fue conocido por su participación en la redacción de la Constitución de 1945 en Ecuador. ¿Hay algo más en lo que pueda ayudarte?
Human: Cual fue la relacion de Jose

'José María Sarasti y Eloy Alfaro tuvieron una relación política compleja. Ambos fueron prominentes figuras en la historia política de Ecuador, pero tuvieron diferencias ideológicas y disputas personales. Sarasti era miembro del Partido Conservador, mientras que Alfaro lideraba el Partido Liberal Radical. A pesar de sus diferencias, Sarasti y Alfaro colaboraron en algunos momentos. Por ejemplo, Sarasti fue Ministro de Gobierno durante el gobierno de Alfaro en 1906. Sin embargo, también hubo momentos de conflicto entre ellos, especialmente durante la Revolución Liberal de 1911, cuando Sarasti se opuso a Alfaro y apoyó a los conservadores. En resumen, su relación fue compleja y estuvo marcada por la política y las diferencias ideológicas. ¿Hay algo más en lo que pueda ayudarte?'

In [69]:
conversation.predict(input='Calificas como positiva o negativa la gestion de Jose Maria Sarasti en Ecuador')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola quien fue Jose Maria Sarasti en Ecuador
AI: ¡Hola! José María Sarasti fue un destacado político ecuatoriano. Nació el 19 de marzo de 1899 en Guayaquil, Ecuador, y falleció el 6 de julio de 1976 en la misma ciudad. Sarasti fue un miembro prominente del Partido Conservador y ocupó varios cargos políticos a lo largo de su carrera. Fue elegido como diputado en varias ocasiones y también se desempeñó como Ministro de Gobierno y Ministro de Relaciones Exteriores en diferentes períodos. Además, Sarasti fue conocido por su participación en la redacción de la Constitución de 1945 en Ecuador. ¿Hay algo más en lo que pueda ayudarte?
Human: Cual fue la relacion de Jose

'Como inteligencia artificial, no tengo la capacidad de emitir juicios de valor sobre la gestión de una persona. La evaluación de la gestión de José María Sarasti en Ecuador es subjetiva y depende de diferentes perspectivas y opiniones. Algunos podrían considerar su gestión como positiva debido a sus logros políticos y su participación en la redacción de la Constitución de 1945. Sin embargo, otros podrían tener una opinión negativa debido a diferencias ideológicas o a eventos específicos durante su carrera política. En última instancia, la evaluación de su gestión es una cuestión de opinión personal. ¿Hay algo más en lo que pueda ayudarte?'

In [70]:
conversation.predict(input='Podrias decirme si Jose Maria Sarasti vivio en algun punto en Pujili Cotopaxi')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Cual fue la relacion de Jose Maria Sarasti con Eloy Alfaro
AI: José María Sarasti y Eloy Alfaro tuvieron una relación política compleja. Ambos fueron prominentes figuras en la historia política de Ecuador, pero tuvieron diferencias ideológicas y disputas personales. Sarasti era miembro del Partido Conservador, mientras que Alfaro lideraba el Partido Liberal Radical. A pesar de sus diferencias, Sarasti y Alfaro colaboraron en algunos momentos. Por ejemplo, Sarasti fue Ministro de Gobierno durante el gobierno de Alfaro en 1906. Sin embargo, también hubo momentos de conflicto entre ellos, especialmente durante la Revolución Liberal de 1911, cuando Sarasti se opuso 

'No tengo información específica sobre si José María Sarasti vivió en algún momento en Pujilí, Cotopaxi. Mi conocimiento se basa en datos generales y no tengo acceso a información detallada sobre la vida personal de individuos específicos. Sería recomendable buscar fuentes más especializadas o consultar con expertos en la historia de Ecuador para obtener información más precisa sobre la residencia de José María Sarasti. ¿Hay algo más en lo que pueda ayudarte?'

In [71]:
conversation.predict(input='Como es el clima en Ecuador')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Calificas como positiva o negativa la gestion de Jose Maria Sarasti en Ecuador
AI: Como inteligencia artificial, no tengo la capacidad de emitir juicios de valor sobre la gestión de una persona. La evaluación de la gestión de José María Sarasti en Ecuador es subjetiva y depende de diferentes perspectivas y opiniones. Algunos podrían considerar su gestión como positiva debido a sus logros políticos y su participación en la redacción de la Constitución de 1945. Sin embargo, otros podrían tener una opinión negativa debido a diferencias ideológicas o a eventos específicos durante su carrera política. En última instancia, la evaluación de su gestión es una cuestión d

'Ecuador tiene una gran diversidad climática debido a su ubicación geográfica y su topografía variada. En general, el clima en Ecuador se puede dividir en cuatro regiones principales: la costa, la sierra, la región amazónica y las Islas Galápagos.\n\nEn la costa, el clima es tropical y húmedo, con temperaturas promedio que oscilan entre los 25°C y 31°C. La temporada de lluvias va de diciembre a mayo, mientras que la temporada seca va de junio a noviembre.\n\nEn la sierra, el clima varía según la altitud. En las tierras altas, como Quito, el clima es templado y primaveral durante todo el año, con temperaturas promedio que oscilan entre los 15°C y 25°C. Sin embargo, en las zonas más altas de la sierra, como los páramos, las temperaturas pueden descender por debajo de los 0°C.\n\nEn la región amazónica, el clima es tropical y húmedo durante todo el año, con temperaturas promedio que osc'

In [72]:
conversation.predict(input='Dime algo de Pujili')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Podrias decirme si Jose Maria Sarasti vivio en algun punto en Pujili Cotopaxi
AI: No tengo información específica sobre si José María Sarasti vivió en algún momento en Pujilí, Cotopaxi. Mi conocimiento se basa en datos generales y no tengo acceso a información detallada sobre la vida personal de individuos específicos. Sería recomendable buscar fuentes más especializadas o consultar con expertos en la historia de Ecuador para obtener información más precisa sobre la residencia de José María Sarasti. ¿Hay algo más en lo que pueda ayudarte?
Human: Como es el clima en Ecuador
AI: Ecuador tiene una gran diversidad climática debido a su ubicación geográfica y su topo

'Pujilí es una ciudad ubicada en la provincia de Cotopaxi, en Ecuador. Se encuentra a una altitud de aproximadamente 2,800 metros sobre el nivel del mar. Pujilí es conocida por su rica cultura indígena y su tradición agrícola.\n\nLa ciudad cuenta con varios atractivos turísticos, como la Iglesia Matriz de Pujilí, que es un importante sitio histórico y arquitectónico. También puedes visitar el Mercado de Pujilí, donde encontrarás una gran variedad de productos locales, como frutas, verduras y artesanías.\n\nAdemás, Pujilí es famosa por sus festividades tradicionales, como la Fiesta de la Mama Negra, que se celebra en septiembre. Durante esta festividad, se realizan desfiles coloridos y se llevan a cabo representaciones teatrales que muestran la mezcla de las culturas indígena y española.\n\nEn cuanto a la economía, Pujilí se basa principalmente en la agricultura y la ganadería. Los principales cultivos incluyen maíz'

3. Conversation Summary Memory

This tools avoids to save all data, in this case, it is only saved a summary

In [74]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain import OpenAI 

c:\Users\LP00035\anaconda3\envs\openai\lib\site-packages\langchain\__init__.py:24: UserWarning: Importing OpenAI from langchain root module is no longer supported.
  warnings.warn(


In [78]:
# it must be used a LLM for the summary
summary_memory = ConversationSummaryMemory(llm=OpenAI(openai_api_key=OPENAI_KEY))

In [79]:
conversation =  ConversationChain(memory=summary_memory, llm=llm, verbose=True)

In [80]:
conversation.predict(input='Hi which football team is the most popular in Spain?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi which football team is the most popular in Spain?
AI:

> Finished chain.


'Hi! The most popular football team in Spain is Real Madrid. They have a huge fan base and have won numerous domestic and international titles.'

In [81]:
conversation.predict(input='Why not Barcelona is not the most popular team in Spain?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The AI also states that the most popular football team in Spain is Real Madrid, with a huge fan base and many domestic and international titles.
Human: Why not Barcelona is not the most popular team in Spain?
AI:

> Finished chain.


"Barcelona is indeed a very popular football team in Spain, but Real Madrid has historically had a larger fan base and has won more domestic and international titles. Real Madrid has a rich history and has consistently been one of the most successful teams in Spanish football. However, it's important to note that popularity can vary depending on personal preferences and regional loyalties."

In [82]:
conversation.predict(input='Is Mallorca a team from Las Palmas?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The AI also states that the most popular football team in Spain is Real Madrid, with a huge fan base and many domestic and international titles. The AI explains that Barcelona is also a very popular team, but Real Madrid has historically had a larger fan base and has won more titles. It is noted that popularity can vary depending on personal preferences and regional loyalties.
Human: Is Mallorca a team from Las Palmas?
AI:

> Finished chain.


'No, Mallorca is not a team from Las Palmas. Mallorca is a football team from the island of Mallorca, which is part of the Balearic Islands in Spain. Las Palmas, on the other hand, is a football team from the city of Las Palmas de Gran Canaria, which is located in the Canary Islands. Both teams compete in the Spanish football league system, but they are from different regions.'

In [83]:
conversation.predict(input='Is Celta de Vigo from la Rioja?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
 
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The AI also states that the most popular football team in Spain is Real Madrid, with a huge fan base and many domestic and international titles. It is noted that popularity can vary depending on personal preferences and regional loyalties. The human then asks if Mallorca is a team from Las Palmas, to which the AI responds that Mallorca is a team from the island of Mallorca in the Balearic Islands, while Las Palmas is a team from the city of Las Palmas de Gran Canaria in the Canary Islands.
Huma

'No, Celta de Vigo is not from La Rioja. Celta de Vigo is a football team from the city of Vigo, which is located in the region of Galicia in northwest Spain. La Rioja, on the other hand, is an autonomous community located in northern Spain.'

4. Conversation Summary Buffer Memory

This kind of memory is suitable when you have small conversations, in this case, you have the option to limit the range of the summary

In [ ]:
!pip install tiktoken

In [96]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

In [97]:
memory = ConversationSummaryBufferMemory(llm=OpenAI(openai_api_key=OPENAI_KEY), max_token_limit=10)

In [98]:
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [99]:
conversation.predict(input = 'What is a llama')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is a llama
AI:

> Finished chain.


'A llama is a domesticated South American camelid that is commonly used as a pack animal and for its wool. It is native to the Andes Mountains and is closely related to the alpaca. Llamas have a long neck, a slender body, and a camel-like face. They are known for their soft and warm wool, which is highly valued for its quality. Llamas are also known for their gentle and friendly nature, making them popular as pets and therapy animals. They are herbivores and primarily feed on grass and other vegetation. Llamas are social animals and live in herds, typically led by a dominant male called a "stud." They have been domesticated for thousands of years and have been used by humans for transportation, wool production, and even as guard animals for livestock.'

In [100]:
conversation.predict(input = 'Do they live in the coastal regions?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asked what a llama is and the AI responded that it is a domesticated South American camelid that is commonly used as a pack animal and for its wool. It is native to the Andes Mountains and is closely related to the alpaca. Llamas have a long neck, a slender body, and a camel-like face. They are known for their soft and warm wool, which is highly valued for its quality, and for their gentle and friendly nature, making them popular as pets and therapy animals. They are herbivores and primarily feed on grass and other vegetation, and live in herds led by a dominant male. They have been domesticated for thousands of years and have been used by humans for

'No, llamas are primarily found in the Andes Mountains of South America, which are not coastal regions. They are adapted to the high altitudes and cold climates of the mountainous regions. However, llamas have been introduced to other parts of the world, including coastal regions, where they can adapt to different climates and environments.'

In [101]:
conversation.predict(input = 'What are differences between llama an vicuña?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asked what a llama is and the AI responded that it is a domesticated South American camelid that is commonly used as a pack animal and for its wool. It is native to the Andes Mountains and is closely related to the alpaca. Llamas have a long neck, a slender body, and a camel-like face, are herbivores, and are known for their soft and warm wool, gentle and friendly nature, making them popular as pets and therapy animals. They have been domesticated for thousands of years and have been used by humans for transportation, wool production, and even as guard animals for livestock. The AI then clarified that llamas are primarily found in the Andes Mountains

'Llamas and vicuñas are both South American camelids, but there are several differences between them. Firstly, their physical appearance is distinct. Llamas are larger and have a more robust build compared to vicuñas, which are smaller and more delicate. Llamas have a long neck, a slender body, and a camel-like face, while vicuñas have a shorter neck, a compact body, and a more deer-like face.\n\nAnother difference is their natural habitat. Llamas are primarily found in the Andes Mountains of South America, whereas vicuñas inhabit the high-altitude regions of the Andes, particularly in Peru, Bolivia, Chile, and Argentina. Vicuñas are specifically adapted to live in the harsh conditions of the high mountains, where temperatures can be extremely cold and oxygen levels are lower.\n\nTheir wool is also different. Llamas have a thicker and coarser wool, which is commonly used for making clothing, blankets, and other textiles. On the other hand, vicuñas produce one of the finest and most exp

In [102]:
conversation.predict(input = 'Is a guanaco the same as a llama?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asked what a llama is and the AI responded that it is a domesticated South American camelid that is commonly used as a pack animal and for its wool. It is native to the Andes Mountains and is closely related to the alpaca. Llamas have a long neck, a slender body, and a camel-like face, are herbivores, and are known for their soft and warm wool, gentle and friendly nature, making them popular as pets and therapy animals. They have been domesticated for thousands of years and have been used by humans for transportation, wool production, and even as guard animals for livestock. The AI then clarified that llamas are primarily found in the Andes Mountains

'No, a guanaco is not the same as a llama. While both are South American camelids, they are different species. Guanacos are wild animals and are found in the wild in the Andes Mountains of South America, whereas llamas are domesticated and can be found in various parts of the world. Guanacos are smaller than llamas and have a slender body, long legs, and a short tail. They have a similar camel-like face to llamas but lack the distinctive hump on their back. Guanacos have a different natural habitat and behavior compared to llamas. They are adapted to live in the high altitudes and cold climates of the Andes Mountains, and they are known for their agility and ability to navigate rocky terrains. Guanacos have a different type of wool compared to llamas, which is coarser and less desirable for commercial use.'

In [109]:
conversation.memory.template 

AttributeError: 'ConversationSummaryBufferMemory' object has no attribute 'template'

5. Memory Entity

This memory helps to save information given from users.